In [1]:
import os
from torch.utils.data import Dataset
from torchvision.io import decode_image

class CustomImageDataset(Dataset):
    def __init__(self, img_dir, transform=None, target_transform=None):
        self.img_dir = img_dir
        self.img_files = [f for f in os.listdir(img_dir) if f.endswith(('.png', '.jpg', '.jpeg'))]
        self.transform = transform
        self.target_transform = target_transform
    
    def __len__(self):
        return len(self.img_files)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_files[idx])
        image = decode_image(img_path)
        if self.transform:
            image = self.transform(image)
        return image

In [2]:
from torch.utils.data import random_split
from torch.utils.data import DataLoader
from torchvision import transforms
import torch

# Define transformations to ensure all images are the same size
# decode_image returns a uint8 tensor, so we need to resize and convert to float
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ConvertImageDtype(torch.float),
])

# Pass the transform to the dataset
dataset = CustomImageDataset("Datensatz_Cologne", transform=transform)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
training_data, test_data = random_split(dataset, [train_size, test_size])

train_loader = DataLoader(training_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [3]:
training_data[1]

tensor([[[0.9647, 0.9608, 0.9608,  ..., 0.9569, 0.9608, 0.9608],
         [0.9608, 0.9608, 0.9569,  ..., 0.9569, 0.9608, 0.9608],
         [0.9608, 0.9569, 0.9569,  ..., 0.9569, 0.9608, 0.9608],
         ...,
         [0.9059, 0.9255, 0.9373,  ..., 0.9333, 0.9216, 0.9098],
         [0.8863, 0.9059, 0.9059,  ..., 0.9176, 0.8980, 0.9176],
         [0.7255, 0.7333, 0.7373,  ..., 0.8000, 0.7765, 0.8039]],

        [[0.9608, 0.9608, 0.9569,  ..., 0.9529, 0.9608, 0.9608],
         [0.9608, 0.9569, 0.9529,  ..., 0.9569, 0.9608, 0.9608],
         [0.9608, 0.9529, 0.9529,  ..., 0.9529, 0.9608, 0.9608],
         ...,
         [0.9098, 0.9294, 0.9373,  ..., 0.9333, 0.9216, 0.9098],
         [0.8824, 0.9020, 0.9020,  ..., 0.9176, 0.8980, 0.9176],
         [0.7020, 0.7098, 0.7137,  ..., 0.7922, 0.7686, 0.7922]],

        [[0.9725, 0.9725, 0.9725,  ..., 0.9686, 0.9686, 0.9686],
         [0.9686, 0.9686, 0.9725,  ..., 0.9725, 0.9686, 0.9725],
         [0.9725, 0.9686, 0.9725,  ..., 0.9725, 0.9686, 0.

In [4]:
import torch
import torch.nn as nn

class PatchEmbedding(nn.Module):
  def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
      super().__init__()
      self.patch_size = patch_size
      self.proj = nn.Conv2d(in_channels, embed_dim, kernel_size=patch_size, stride=patch_size)
  def forward(self, x):
    B, C, H, W = x.shape
    x =  self.proj(x).flatten(2).transpose(1, 2)
    return x

In [5]:
class PositionEncoding(nn.Module):
    def __init__(self, embed_dim, seq_len):
        super().__init__()
        self.pos_embed = nn.Parameter(torch.randn(1, seq_len + 1, embed_dim))
# Adjusted for [CLS] token

    def forward(self, x):
        return x + self.pos_embed

In [6]:
class MultiheadAttention(nn.Module):
  # the class inherits from nn.Module and we need to just
  # add the relevant parameters for the MHA class which are
  # embed_dim and num_heads
  def __init__(self, embed_dim, num_heads):
    super().__init__()
    self.attn = nn.MultiheadAttention(embed_dim, num_heads, batch_first=True)
  # now we just need a function to execute the mathematical operation
  def forward(self, x):
    return self.attn(x, x, x)[0]

In [7]:
class TransformerEncoderBlock(nn.Module):
    def __init__(self, embed_dim, num_heads, mlp_dim):
        super().__init__()
        self.attn = MultiheadAttention(embed_dim, num_heads)
        # Renamed self.mlp_dim to self.mlp to match usage in forward()
        self.mlp = nn.Sequential(
                        nn.Linear(embed_dim, mlp_dim), 
                        nn.ReLU(),
                        nn.Linear(mlp_dim, embed_dim)
        )
        self.norm1 = nn.LayerNorm(embed_dim)
        self.norm2 = nn.LayerNorm(embed_dim)
    
    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

In [8]:
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, num_classes=10, embed_dim=768, num_heads=8, depth=6, mlp_dim=1024):
        super().__init__()
        self.patch_embedding = PatchEmbedding(img_size, patch_size, 3, embed_dim)
        self.pos_encoding = PositionEncoding(embed_dim, (img_size // patch_size) ** 2)
        self.transformer_blocks = nn.ModuleList([
            TransformerEncoderBlock(embed_dim, num_heads, mlp_dim) for _ in range(depth)
        ])
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))
        self.mlp_head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B = x.size(0)
        x = self.patch_embedding(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = self.pos_encoding(x)
        for block in self.transformer_blocks:
            x = block(x)
        return self.mlp_head(x[:, 0])

Now for the training

In [ ]:
import torch.optim as optim
from torchvision import datasets, transforms

# Note: We already defined 'transform' and 'model' above, reusing them here.
# Ensure 'model' is on the correct device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VisionTransformer()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
model.to(device)

# Training loop
for epoch in range(5):  # Train for 5 epochs
    model.train()
    running_loss = 0.0
    # Since dataset only returns images, we only unpack 'inputs'
    for inputs in train_loader: 
        inputs = inputs.to(device) 
        
        # Create dummy labels for now since we don't have real ones
        # Assuming 10 classes as per VisionTransformer(num_classes=10)
        labels = torch.zeros(inputs.size(0), dtype=torch.long).to(device) 
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(train_loader)}")